## Amazon movie reviews data cleanup
Using spark for pre-processing

In [1]:
import pyspark as ps    # for the pyspark suite
spark = (ps.sql.SparkSession
         .builder
         .master('local[4]')
         .appName('moviemood')
         .getOrCreate()
        )

In [2]:
data_path = '../../datasets/'
file_name = 'reviews_Movies_and_TV.json.gz'

In [3]:
amazon_reviews_df = spark.read.json(data_path + file_name)

In [4]:
amazon_reviews_df.cache()
amazon_reviews_df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [5]:
total_comment_count = amazon_reviews_df.count()

In [6]:
total_comment_count

4607047

### Data sampling for development

In [7]:
dev_sample_fraction = 0.02

In [8]:
reviews_spark_df = amazon_reviews_df.sample(withReplacement=False, fraction=0.02)

In [9]:
reviews_df = reviews_spark_df.toPandas()

In [10]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91869 entries, 0 to 91868
Data columns (total 9 columns):
asin              91869 non-null object
helpful           91869 non-null object
overall           91869 non-null float64
reviewText        91869 non-null object
reviewTime        91869 non-null object
reviewerID        91869 non-null object
reviewerName      91524 non-null object
summary           91869 non-null object
unixReviewTime    91869 non-null int64
dtypes: float64(1), int64(1), object(7)
memory usage: 6.3+ MB


In [11]:
reviews_df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0001526863,"[0, 0]",5.0,"Attractive, memorable video featuring songs th...","09 27, 2009",A1TN0V94A3ECSH,Karilyn S. Eastvold,Hide 'Em in Your Heart,1254009600
1,0005004101,"[2, 3]",4.0,See also: Jehovah's Witnesses: A Non-'Prophet'...,"07 2, 2006",A3KROYMQ61M7A,Rich The Reviewer,Very Fine Jeremiah Films production of this pe...,1151798400
2,0005019281,"[0, 0]",5.0,"This has not been on TV, that I know of in man...","12 14, 2012",A1ZAQHMCCT297C,Holly Anne Romanoli,So happy I was able to get this,1355443200
3,0005019281,"[0, 0]",5.0,I love An American Christmas Carol with Henry ...,"01 4, 2014",A38JFF6SGZEO37,Lauren,The Best Film about Scrooge!,1388793600
4,0005019281,"[4, 4]",5.0,"One of the best ""Scrooge"" movies, a very creat...","01 17, 2007",A22J3OCEN0J191,Steve Gick,An American Chrismas Carol- Henry Winkler,1168992000


### Filter out comments that may be relative to the support (DVD, edition...)

In [33]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

words_to_remove = ['dvd', 'vhs','edition', 'blue-ray', 'blueray', 'blu-ray', 'bluray', 'price']

In [34]:
words_to_remove_re = '(' + '|'.join(words_to_remove) + ')'

In [35]:
import warnings
warnings.simplefilter('ignore')
text_removed = reviews_df[reviews_df['reviewText'].str.contains(words_to_remove_re, case=False)]
warnings.simplefilter('default')

In [36]:
text_removed.describe()

,overall,unixReviewTime
count,21672.000000,2.167200e+04
mean,4.183924,1.252101e+09
std,1.262963,1.184108e+08
min,1.000000,8.949312e+08
25%,4.000000,1.167782e+09
50%,5.000000,1.272413e+09
75%,5.000000,1.358575e+09
max,5.000000,1.405987e+09


#### Result: fraction of content removed:

In [37]:
text_removed['reviewText'].count() / (total_comment_count * dev_sample_fraction)

0.23520489372042439

## Quality check on filtered data

In [43]:
warnings.simplefilter('ignore')
relevant_comments = reviews_df[~reviews_df['reviewText'].str.contains(words_to_remove_re, case=False)]
warnings.simplefilter('default')

In [46]:
relevant_comments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70197 entries, 0 to 91868
Data columns (total 9 columns):
asin              70197 non-null object
helpful           70197 non-null object
overall           70197 non-null float64
reviewText        70197 non-null object
reviewTime        70197 non-null object
reviewerID        70197 non-null object
reviewerName      69930 non-null object
summary           70197 non-null object
unixReviewTime    70197 non-null int64
dtypes: float64(1), int64(1), object(7)
memory usage: 5.4+ MB


In [43]:
relevant_comments['reviewText'].head(2) # try head(50) or more

0    Attractive, memorable video featuring songs that are catchy and easy to learn.  Great way to write the Word of God on young hearts.  Works for parents too--they like the songs and learn the verses with little effort.  Children love these videos.                                                         
1    See also: Jehovah's Witnesses: A Non-'Prophet' Organization. Started by 33rd degree Masonic Pyramidologist Chas.Taze Russell, in 1874 as the 'Russel-ites'. These poor people are badly bam-boozled. They need our prayers and th-is video / book (if you can still findit) are a step in the right direction!
Name: reviewText, dtype: object

#### Result: reviews look good
Based on quick review of head(50)

## Keyword-based sentiment analysis

In [54]:
emotion_keywords_path = '../../datasets/'
emotion_keywords = 'emotions-sensor-data-set.zip'
emotions_keywords_df = pd.read_csv(emotion_keywords_path + emotion_keywords)

In [57]:
emotions_keywords_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1104 entries, 0 to 1103
Data columns (total 8 columns):
word        1104 non-null object
disgust     1104 non-null float64
surprise    1104 non-null float64
neutral     1104 non-null float64
anger       1104 non-null float64
sad         1104 non-null float64
happy       1104 non-null float64
fear        1104 non-null float64
dtypes: float64(7), object(1)
memory usage: 69.1+ KB


In [132]:
emotions = list(emotions_keywords_df)[1:]
rel_comments_w_sentiments = relevant_comments.reindex(columns= relevant_comments.columns.tolist() + emotions)

In [133]:
rel_comments_w_sentiments.head(3)

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,disgust,surprise,neutral,anger,sad,happy,fear
0,0001526863,"[0, 0]",5.0,"Attractive, memorable video featuring songs that are catchy and easy to learn. Great way to write the Word of God on young hearts. Works for parents too--they like the songs and learn the verses with little effort. Children love these videos.","09 27, 2009",A1TN0V94A3ECSH,Karilyn S. Eastvold,Hide 'Em in Your Heart,1254009600,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0005004101,"[2, 3]",4.0,"See also: Jehovah's Witnesses: A Non-'Prophet' Organization. Started by 33rd degree Masonic Pyramidologist Chas.Taze Russell, in 1874 as the 'Russel-ites'. These poor people are badly bam-boozled. They need our prayers and th-is video / book (if you can still findit) are a step in the right direction!","07 2, 2006",A3KROYMQ61M7A,Rich The Reviewer,Very Fine Jeremiah Films production of this pernicious cult...,1151798400,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0005019281,"[0, 0]",5.0,"This has not been on TV, that I know of in many years but really great performance by Henry Winkler. Thank you.","12 14, 2012",A1ZAQHMCCT297C,Holly Anne Romanoli,So happy I was able to get this,1355443200,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
import re
comment = rel_comments_w_sentiments['reviewText'][8]
comment

"This is a brilliant epic movie, a tour de force, the best Biblical movie I have seen with the exception ofOne Night with the KingStarring Paul Mercurio in the title role, he gives an intelligent and passionate performance of a man who was prepared to die, to keep to the word of his G-D.Ben Kingsley is his usual magnificent self in his role of Potiphar which he plays with panache. Warren Clark played a sterling role as Potiphar's thuggish overseer Domo Ednan, later to be Joseph's aid de camp. A deliciously evil Potiphar's Wife is captured in brilliant detail by Lesley Ann Warren and the actors playing the ten brothers also acquit themselves well staying true to the form of Israelite tribesmanThe movie begins with the purchase of Joseph by Potiphar, and his rise from that of a lowly slave to Potiphar's major domo where he captures the attention of Potiphar's evil wife, when he resists her attention and she accuses him of assaulting her, Potiphar soon knows he is innocent, and he embarks

In [134]:
# TODO: remove stomp words
test_input = 'ability. Ability,abilities'
scores = {key: 0.0 for key in emotions}
print(scores)

{'disgust': 0.0, 'surprise': 0.0, 'neutral': 0.0, 'anger': 0.0, 'sad': 0.0, 'happy': 0.0, 'fear': 0.0}


In [148]:
emotion_keywords_hits = 0
scores = {key: 0.0 for key in emotions}    # {'disgust': 0.0, ...}
for row in emotions_keywords_df.itertuples():
#     print(row)
    word = row.word.strip()  # there are spaces at end
    word_re = r'\b' + re.escape(word) + r'\b'
    match = re.findall(word_re, comment,re.IGNORECASE)
    count = len(match)
    if count > 0:
#         print(match)
        scores['disgust'] += row.disgust
        scores['surprise'] += row.surprise
        scores['neutral'] += row.neutral
        scores['anger'] += row.anger
        scores['sad'] += row.sad
        scores['happy'] += row.happy
        scores['fear'] += row.fear
        emotion_keywords_hits += 1

print(scores)
print(emotion_keywords_hits) 

for emotion in emotions:
    rel_comments_w_sentiments.loc[8, emotion] = scores[emotion]
print(rel_comments_w_sentiments.loc[8])

{'disgust': 0.298169173366, 'surprise': 0.7785688146200002, 'neutral': 0.15035548793999998, 'anger': 0.593098854015, 'sad': 0.449492349956, 'happy': 0.8607997053100002, 'fear': 0.89701848154}
26
asin              0005119367                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [93]:
from compute_emotions import *
#computeEmotions(4)

In [95]:
emotions_keywords_df.iloc[0]

word        ability    
disgust     0.00446429 
surprise    0.0478316  
neutral     0.000637755
anger       0.0235969  
sad         0.0133929  
happy       0.0159439  
fear        0.0401786  
Name: 0, dtype: object

In [94]:
# rel_comments_w_sentiments.transform(computeEmotions, axis=1)

## Isolating Comment helpfulness

In [24]:
helpful = reviews_df['helpful'].map(lambda x: x[0], na_action='ignore')
helpful.describe()

count    91869.000000
mean     2.638866    
std      11.521295   
min      0.000000    
25%      0.000000    
50%      0.000000    
75%      2.000000    
max      1031.000000 
Name: helpful, dtype: float64

In [25]:
not_helpful = reviews_df['helpful'].map(lambda x: x[1], na_action='ignore')
not_helpful.describe()

count    91869.000000
mean     4.093655    
std      14.126499   
min      0.000000    
25%      0.000000    
50%      1.000000    
75%      4.000000    
max      1160.000000 
Name: helpful, dtype: float64

### Note: current filtering is probably too aggressive:'DVD' mostly used to describe the movie
Removing DVD would remove a lot of good comments, as all of the following are appropriate:

In [26]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
text_removed['reviewText'].head(5)

11    I have watched this version of &#34;Joseph&#34; numerous times also on TV and am once again glad to add this one to my collection of Bible DVD's. It's entertainment but for me it's much than that I'm able to receive a better understanding of my Bible it's like a learning process to test me on my Bible knowledge and see how well I know the scriptures, for instance when I'm able to pick something out of the movie that may not be quite true to scripture I go check my Bible and I'm able to learn my scriptures better and my Bible. I believe by watching the DVD over and over and even having different versions you absorb more into your memory and it can be your own individual Bible study. I was really amazed and excited when I saw that you offered many of these DVD's of (The Bible Collection) which are some of my favorites. Hoping to collect them all and other versions plus other Christian DVD's to learn from them for a better understanding of God's true Word.
13    I use the Hemisphere

In [28]:
text_removed = reviews_df[(reviews_df['reviewText'].str.contains('(dvd|vhs)', case=False)) & (helpful > 1)]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


In [29]:
text_removed[['reviewText','helpful']].head(5)

,reviewText,helpful
30,"What with Meryl Streep portraying her in a movie and all, Julia's books and series are getting a monster tidal wave of fresh attention. I wonder how many youngsters (Food Network viewers) will purchase or have purchased for them the classic ""Mastering the Art of French Cooking"" and find it completely overwhelming and not necessarily useful for everyday cooking?I had few cooking skills in the mid 1980s when this book and DVD came out, and I relied on recipes from Fannie Farmer, supermarket recipe cards, and women's magazines. This series elevated my cooking for good. It is French influenced, but simplified in some cases, and there are variations on most recipes which is really useful if you don't have time to run to the supermarket. I have had many variation cookbooks and this is the best. I have not used the dessert section too much (I prefer Pierre Franey for this--fruit salad with Grand Marnier!), but the bread, meat, and veg sections are the best. I recommend ham steaks with peas and mashed potatoes and cornish game hen broiled w/cheese.I am so pleased this is out on DVD. I gathered up the video tapes one-by-one on ebay and this is definitely more affordable!","[31, 34]"
37,"To start off with, I actually have an early release of the dvd, so my stars are based on that, not the cinema.I made it through about six or seven pages on reviews before I decided to write this and from what I see, three subjects usually appear as the reason for bad reviews.The first reason is it is too violent. We all knew this, before we even went to see it. It was all over news, media, etc. We were warned not to take our kids, be prepared for violence, not for the weak of heart. So, if you went expecting scenes of the beach, then that is your own fault. I didn't watch it hoping to see a blood bath, but I knew it was coming.I am a very faithful Protestant, and I do see some Catholic overtones in the story, but that is Mel's interpretation and overall I felt it was pretty accurate. It is interesting to me that all of these people that may or may not have a personal relationship with Jesus, seem to know that if he were here today, he would not support this film.The second reason people complain is because it doesn't tell the full story of Jesus. It isn't supposed to. It is the Passion of the Christ, and passion means suffering. It isn't the story of the Christ or the Gospel of the Christ. It is basically a Passion play put to cinema. He isn't trying to show us the teachings of Jesus, he is showing us the suffering of Jesus. He is showing us what He went through for us. If you see this movie and aren't moved by the sacrifice that He made for all of us to be free, then you don't get it. And if you don't understand and you don't get it, then that is ok, I pray someday that you will.The third thing I pick up on in these reviews is that Mel is using it just for profit. He has made money at ever other movie he has made I am sure, and yet no one ever questions his motives then. Yet, when he comes out with something of a religious nature when we are in a world that requires us to be politically correct, then he was just doing it for money. He put up his own money and I personally hope he makes all the money he can off of it, not that he needs it. So what if there is a director's cut later on? How may different directors editions or box sets of Lord of the Rings are out there? I get tired of having to apologize for what I believe. If someone stands up for an important issue, whatever the hot topic of the week is, then that is their right to voice their opinion and they expect everyone to recieve it and agree. But if I as a Christian stand up for what I believe, then I am a ""Jesus Freak"" or a ""Fanatic"". So if you don't agree with me, that is your right, but if I don't believe what you do, then I am narrow minded?? What happened to my right to my own beliefs?While I am still on my soap box, it is amazing to me that people still think